# CEE6501 — Lecture 7.2

## Introduction to 2D Frame Analysis

## Learning Objectives

By the end of this lecture, you will be able to:

- 

## Agenda

Part 1 — Roadmap: truss → beam → 3D frame  
Part 2 — 

## Part 8 - Local to Global Transformation

**Note**: See Lecture 3.2 slides (Trusses) for detaled information on this.

### Generic Truss Element in a Structure

<div style="display:flex; gap:1.2rem; align-items:center;">
  <div style="flex:0.80;">
    <figure style="margin:0; text-align:center; display:flex; flex-direction:column;">
      <img src="assets/L2_FrameGlobal.png" style="width:100%; height:auto;">
      <figcaption style="font-size:0.75em; margin-top:0.4em;"></figcaption>
    </figure>
  </div>
</div>



### Local/Global Perspective

<div style="display:flex; gap:1.5rem; align-items:center;">
  <div style="flex:0.95;">
    <figure style="margin:0; text-align:center; display:flex; flex-direction:column;">
      <img src="assets/L2_Local2Global.png" style="width:80%; height:auto;">
      <figcaption style="font-size:0.75em; margin-top:0.4em;">
        <!-- Local (left) vs. global (right) descriptions of the same member -->
      </figcaption>
    </figure>
  </div>
</div>


- **Local coordinate system (left):** forces $\boldsymbol{Q}$, displacements $\boldsymbol{u}$
- **Global coordinate system (right):** forces $\boldsymbol{F}$, displacements $\boldsymbol{v}$

### Global ($\boldsymbol{F}$) → Local ($\boldsymbol{Q}$) Force Transformation
Use trigonometric relationships to resolve global forces into the local coordinate system.

At node $i$:

$$
\begin{aligned}
Q_1 &= F_1 \cos \theta + F_2 \sin \theta \\
Q_2 &= -F_1 \sin \theta + F_2 \cos \theta \\
Q_3 &= F_3
\end{aligned}
$$

The local and global $z$-axes are aligned (out of plane), therefore the bending moment is unchanged:

$$
Q_3 = F_3
$$

At node, $j$:

$$
\begin{aligned}
Q_4 &= F_4 \cos \theta + F_5 \sin \theta \\
Q_5 &= -F_4 \sin \theta + F_5 \cos \theta \\
Q_6 &= F_6
\end{aligned}
$$

### Global ($\boldsymbol{F}$) → Local ($\boldsymbol{Q}$) Force Transformation

- Local element forces $\boldsymbol{Q}$ are obtained by **rotating** global nodal forces $\boldsymbol{F}$ into the member’s local coordinate system  
- Each node contributes a $3 \times 3$ transformation block:
  - Translational DOFs → **rotated by $\theta$**
  - Rotational DOF → **invariant** (local and global $z$-axes are aligned)

$$
\boldsymbol{Q} = \boldsymbol{T}\,\boldsymbol{F}
$$

$$
\begin{Bmatrix}Q_1\\Q_2\\Q_3\\Q_4\\Q_5\\Q_6\end{Bmatrix}
=
\underbrace{
\begin{bmatrix}
\cos\theta & \sin\theta & 0 & 0 & 0 & 0\\
-\sin\theta & \cos\theta & 0 & 0 & 0 & 0\\
0 & 0 & 1 & 0 & 0 & 0\\
0 & 0 & 0 & \cos\theta & \sin\theta & 0\\
0 & 0 & 0 & -\sin\theta & \cos\theta & 0\\
0 & 0 & 0 & 0 & 0 & 1
\end{bmatrix}
}_{\boldsymbol{T}}
\begin{Bmatrix}F_1\\F_2\\F_3\\F_4\\F_5\\F_6\end{Bmatrix}
$$

**Note:** This is identical to the truss transformation for translational DOFs; the only addition is the rotational DOF (unchanged). See truss lecture for details on **direction cosines**.

### Global ($\boldsymbol{v}$) → Local ($\boldsymbol{u}$) Displacements

- Nodal displacements are transformed using the **same rotation matrix** as forces
- Displacements and forces transform identically because they are defined along the **same directions**

$$
\boldsymbol{u} = \boldsymbol{T}\boldsymbol{v}
$$


### Local ($\boldsymbol{Q}$) → Global ($\boldsymbol{F}$) Force

- This is the **reverse of the global → local process**
- Local member forces or displacements are **rotated back** into the global $(X,Y)$ directions
$$
\boldsymbol{F} = \boldsymbol{T}^{\mathsf{T}}\boldsymbol{Q}
$$

### Local ($\boldsymbol{u}$) → Global ($\boldsymbol{v}$) Displacements

- This is the **reverse of the global → local process**
- Local member forces or displacements are **rotated back** into the global $(X,Y)$ directions
$$
\boldsymbol{v} = \boldsymbol{T}^{\mathsf{T}}\boldsymbol{u}
$$

## Wrap-Up

In this lecture, we:


Next lecture:
 